In [1]:
# Assistant API(OpenAI)
# Agent와 비슷한 기능 -> Langchain 없이 OpenAI Python 패키지로 가능
"""
1. Assistant
 : Tool(Instructino) 과 OpenAI Model 로 조합된 AI Model
2. Thread
 : User와 Assistant가 주고 받는 메시지의 모음
 : User가 Message를 보내면 Thread에 전달되야 함.
 : 별도 메모리 관리를 할 것없이 OpenAI가 이를 관리하여 Message들을 Thread에 관리됨(Model의 context memory에 맞도록 history 관리)
3. Messages
 : User Message
4. Run 
 : Thread의 내용을 Model을 통해서 실행
"""
from langchain.tools import DuckDuckGoSearchResults
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchResults()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


get_ticker({"company_name":"Cloudflare"})

"[snippet: Real-time Cloudflare stock price (NYSE: NET) plus other valuable data points like day range, year, stock analyst insights, related news and more., title: NET Stock - Cloudflare Stock Price - NYSE: NET | Morningstar, link: https://www.morningstar.com/stocks/xnys/net/quote], [snippet: Get a real-time Cloudflare, Inc. (NET) stock price quote with breaking news, financials, statistics, charts and more. Get a real-time Cloudflare, Inc. (NET) stock price quote with breaking news, financials, statistics, charts and more. ... Ticker Symbol NET. Full Company Profile. Financial Performance. In 2023, Cloudflare's revenue was $1.30 ..., title: Cloudflare, Inc. (NET) Stock Price, Quote & News - Stock Analysis, link: https://stockanalysis.com/stocks/net/], [snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: h

In [2]:
import openai as client

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-0125-preview", # https://platform.openai.com/playground?mode=assistant : Model List
    tools=functions,
)

assistant

Assistant(id='asst_5W5rXnfx64ayBqbDXTkQTUEr', created_at=1712824706, description=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', metadata={}, model='gpt-4-0125-preview', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker']}), type='function'), FunctionTool(function=FunctionDefinition(n